In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import wrangle_zillow

In [2]:
df = wrangle_zillow.wrangle_zillow()

Using cached data


In [3]:
df.shape

(77380, 61)

In [4]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 61 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          77380 non-null  int64  
 1   parcelid                    77380 non-null  int64  
 2   basementsqft                50 non-null     float64
 3   bathroom                    77380 non-null  float64
 4   bedroom                     77380 non-null  float64
 5   condition                   49671 non-null  float64
 6   calculatedbathnbr           76771 non-null  float64
 7   decktypeid                  614 non-null    float64
 8   finishedfloor1squarefeet    6023 non-null   float64
 9   square_feet                 77184 non-null  float64
 10  finishedsquarefeet12        73748 non-null  float64
 11  finishedsquarefeet13        41 non-null     float64
 12  finishedsquarefeet15        3009 non-null   float64
 13  finishedsquarefeet50        602

,id,parcelid,basementsqft,bathroom,bedroom,condition,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,square_feet,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,tax_value,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,7.738000e+04,7.738000e+04,50.000000,77380.000000,77380.000000,49671.000000,76771.000000,614.0,6023.000000,77184.000000,...,17560.000000,172.0,7.726800e+04,7.737900e+04,77380.0,7.737800e+04,77375.000000,2886.000000,7.713600e+04,77380.000000
mean,1.495126e+06,1.300715e+07,679.720000,2.299134,3.053489,6.534638,2.316871,66.0,1366.512535,1785.229465,...,1.434738,1.0,1.893160e+05,4.901375e+05,2016.0,3.010969e+05,5995.566592,14.087318,6.049674e+13,0.016628
std,8.609057e+05,3.481368e+06,689.703546,0.996657,1.139103,1.721933,0.979761,0.0,671.308125,954.051796,...,0.544569,0.0,2.300886e+05,6.534477e+05,0.0,4.925990e+05,7622.887460,2.185663,1.535252e+12,0.170190
min,3.490000e+02,1.071186e+07,38.000000,0.000000,0.000000,1.000000,1.000000,66.0,44.000000,128.000000,...,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,7.520500e+05,1.153830e+07,273.000000,2.000000,2.000000,6.000000,2.000000,66.0,955.000000,1182.000000,...,1.000000,1.0,8.426500e+04,2.070000e+05,2016.0,8.550400e+04,2715.585000,14.000000,6.037311e+13,-0.024377
50%,1.497870e+06,1.253155e+07,515.000000,2.000000,3.000000,6.000000,2.000000,66.0,1257.000000,1543.000000,...,1.000000,1.0,1.364995e+05,3.589760e+05,2016.0,2.033835e+05,4450.720000,15.000000,6.037603e+13,0.006627
75%,2.240480e+06,1.421184e+07,796.500000,3.000000,4.000000,8.000000,3.000000,66.0,1615.000000,2113.000000,...,2.000000,1.0,2.187875e+05,5.690030e+05,2016.0,3.668022e+05,6927.800000,15.000000,6.059042e+13,0.039204
max,2.982274e+06,1.676893e+08,3560.000000,18.000000,16.000000,12.000000,18.000000,66.0,6912.000000,35640.000000,...,6.000000,1.0,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


### Remove any properties that are not single unit. Don't want to lose all the nans

In [5]:
df.unitcnt[(df.unitcnt==1)|(df.unitcnt.isna())].value_counts(dropna=False)

1.0    47292
NaN    26818
Name: unitcnt, dtype: int64

In [6]:
# Filter out anything other than unit count = 1 and nans
df = df[(df.unitcnt==1)|(df.unitcnt.isna())]

In [7]:
# Keep properties that should be single units
properties_to_keep = ['Single Family Residential','Condominum','Mobile Home','Manufactured, Modular, Prefabricated Homes','Residential General','Townhouse',np.nan]
df = df[df.apply(lambda row: row.propertylandusedesc in properties_to_keep, axis=1)]
df.propertylandusedesc.value_counts(dropna=False)

Single Family Residential                     52291
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              34
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

### Functions to drop rows or columns based on percent of values that are missing

In [8]:
wrangle_zillow.nulls_by_column(df)

,n_rows_missing,pct_rows_missing
buildingclassdesc,52463,1.000000
finishedsquarefeet15,52463,1.000000
finishedsquarefeet13,52422,0.999218
basementsqft,52416,0.999104
storydesc,52416,0.999104
...,...,...
propertycountylandusecode,0,0.000000
parcelid,0,0.000000
roomcnt,0,0.000000
rawcensustractandblock,0,0.000000


# Takeaways from nulls by column:
- For some (especially binary or few categories) may be able to assume null means 0:
    - check nunique and fillna with 0
- May want to see if any specific category is overrepresenting with nulls. Should fill these nas with a value (like 'unavailable') so the rest of the information is not dropped
    - Like one county didn't have any condition information
- After that may want to drop columns with >90% nulls

In [9]:
wrangle_zillow.nulls_by_row(df)

,num_cols_missing,pct_cols_missing,num_rows
0,28,0.459016,15862
1,29,0.475410,12517
2,27,0.442623,7936
3,26,0.426230,5919
4,30,0.491803,4460
5,25,0.409836,2909
6,31,0.508197,1071
7,24,0.393443,534
8,23,0.377049,479
9,22,0.360656,261


# Takeaway from nulls by row
- Should fix issues from nulls by column first - this might lower the nulls by row values
- seems like removing any rows with >60% of missing columns would affect fairly minimal amount

In [10]:
df_nulls_removed = wrangle_zillow.handle_missing_values(df, prop_required_column=0.3, prop_required_row=0.00002)

(52463, 61)  original shape
(52463, 59)  shape after dropping columns with prop required rows below theshold
(52463, 59)  shape after dropping rows with prop required columns below threshold


In [11]:
wrangle_zillow.nulls_by_column(df_nulls_removed)

,n_rows_missing,pct_rows_missing
finishedsquarefeet13,52422,0.999218
basementsqft,52416,0.999104
storydesc,52416,0.999104
yardbuildingsqft26,52399,0.998780
architecturalstyledesc,52393,0.998666
typeconstructiondesc,52387,0.998551
fireplaceflag,52382,0.998456
finishedsquarefeet6,52299,0.996874
decktypeid,52072,0.992547
pooltypeid10,52020,0.991556


In [12]:
wrangle_zillow.nulls_by_row(df_nulls_removed)

,num_cols_missing,pct_cols_missing,num_rows
0,26,0.440678,15862
1,27,0.457627,12517
2,25,0.423729,7936
3,24,0.406780,5919
4,28,0.474576,4460
5,23,0.389831,2909
6,29,0.491525,1071
7,22,0.372881,534
8,21,0.355932,479
9,20,0.338983,261


### Many of the null values remaining are the result of the binary case of 0 or none - such as "hashottuborspa." The only option for many of these are either have it or not, where the "not" option is NaN. Better to fill these in as 0 or None rather than drop them outright. 

In [13]:
for col in df_nulls_removed.columns:
    if df_nulls_removed[col].nunique() == 1:
        print(col)
        print(df_nulls_removed[col].value_counts(dropna=False))
        df_nulls_removed[col] = df_nulls_removed[col].fillna('None')

decktypeid
NaN     52072
66.0      391
Name: decktypeid, dtype: int64
hashottuborspa
NaN    50949
1.0     1514
Name: hashottuborspa, dtype: int64
pools
NaN    41387
1.0    11076
Name: pools, dtype: int64
pooltypeid10
NaN    52020
1.0      443
Name: pooltypeid10, dtype: int64
pooltypeid2
NaN    51392
1.0     1071
Name: pooltypeid2, dtype: int64
pooltypeid7
NaN    42474
1.0     9989
Name: pooltypeid7, dtype: int64
unitcnt
1.0    33802
NaN    18661
Name: unitcnt, dtype: int64
fireplaceflag
NaN    52382
1.0       81
Name: fireplaceflag, dtype: int64
assessmentyear
2016.0    52463
Name: assessmentyear, dtype: int64
taxdelinquencyflag
NaN    50393
Y       2070
Name: taxdelinquencyflag, dtype: int64
storydesc
NaN         52416
Basement       47
Name: storydesc, dtype: int64


In [14]:
wrangle_zillow.nulls_by_column(df_nulls_removed)

,n_rows_missing,pct_rows_missing
finishedsquarefeet13,52422,0.999218
basementsqft,52416,0.999104
yardbuildingsqft26,52399,0.998780
architecturalstyledesc,52393,0.998666
typeconstructiondesc,52387,0.998551
finishedsquarefeet6,52299,0.996874
poolsizesum,51598,0.983512
yardbuildingsqft17,50511,0.962793
taxdelinquencyyear,50393,0.960544
finishedfloor1squarefeet,48085,0.916551


### With the binary cases handled, now moving onto those with more categories. 
- For many of the columns with "desc" in the name, should replace NaN with not-specified
- If cnt or number in the name, probably safe to fill nan with 0


In [15]:
for col in df_nulls_removed.columns:
    if 'desc' in col:
        # print(df[col].value_counts(dropna=False))
        df_nulls_removed[col] = df_nulls_removed[col].fillna('Not Specified')
    elif 'cnt' in col:
        df_nulls_removed[col] = df_nulls_removed[col].fillna(0)
    elif 'number' in col:
        df_nulls_removed[col] = df_nulls_removed[col].fillna(0)

In [16]:
for col in df_nulls_removed.columns:
    if (df_nulls_removed[col].nunique() < 15) & (df_nulls_removed[col].nunique() > 2) & (df_nulls_removed[col].isna().sum()>0):
        print(col)
        print(df_nulls_removed[col].value_counts(dropna=False))
        # df[col] = df[col].fillna('None')

condition
NaN     18767
6.0     10268
8.0      8250
4.0      8131
7.0      3436
5.0      1502
9.0      1122
11.0      515
10.0      230
3.0       149
12.0       80
1.0         8
2.0         5
Name: condition, dtype: int64
finishedsquarefeet13
NaN       52422
1440.0       18
1344.0        5
1536.0        3
1488.0        3
1152.0        2
1392.0        2
1248.0        2
1056.0        1
1300.0        1
1560.0        1
1512.0        1
1060.0        1
1200.0        1
Name: finishedsquarefeet13, dtype: int64
garage
NaN     34439
2.0     14814
1.0      2198
3.0       623
4.0       253
0.0        65
5.0        45
6.0        13
7.0         3
10.0        3
8.0         2
9.0         2
11.0        1
13.0        1
14.0        1
Name: garage, dtype: int64
threequarterbathnbr
NaN    45747
1.0     6637
2.0       70
3.0        8
7.0        1
Name: threequarterbathnbr, dtype: int64
taxdelinquencyyear
NaN     50393
15.0     1123
14.0      561
13.0      126
12.0       82
11.0       79
10.0       47
9.0   

### Remaining columns must be dealt with individually
- garage and threequarterbathnbr: fill nas with 0
- garagetotalsqft: no garage
- basementsqft: no basement information
- poolsizesum: no pool
- taxdelinquencyyear: fill with "Assumed not delinquent"
- condition: not available
- yardbuildingsqft17: no patio information
- yardbuildingsqft26: no yard building
- drop finishedsquarefeet13,50,6 and finishedfloor1squarefeet. Drop finishedsquarefeet12 - looks to be equal to square footage
- drop calculatedbathnbr, redundant

In [17]:
df_nulls_removed['garage'] = df_nulls_removed['garage'].fillna(0)
df_nulls_removed['garagetotalsqft'] = df_nulls_removed['garagetotalsqft'].fillna('No garage')
df_nulls_removed['poolsizesum'] = df_nulls_removed['poolsizesum'].fillna('No pool')
df_nulls_removed['basementsqft'] = df_nulls_removed['basementsqft'].fillna('No basement information')
df_nulls_removed['threequarterbathnbr'] = df_nulls_removed['threequarterbathnbr'].fillna(0)
df_nulls_removed['taxdelinquencyyear'] = df_nulls_removed['taxdelinquencyyear'].fillna("Assumed Not Delinquent")
df_nulls_removed['condition'] = df_nulls_removed['condition'].fillna("Not available")
df_nulls_removed['yardbuildingsqft17'] = df_nulls_removed['yardbuildingsqft17'].fillna("No Patio Information")
df_nulls_removed['yardbuildingsqft26'] = df_nulls_removed['yardbuildingsqft26'].fillna("No Yard Building")
df_nulls_removed = df_nulls_removed.drop(columns = ['calculatedbathnbr','finishedsquarefeet13','finishedsquarefeet50','finishedsquarefeet6','finishedsquarefeet12','finishedfloor1squarefeet'])

In [18]:
for col in df_nulls_removed.columns:
    if df_nulls_removed[col].isna().sum()>0:
        print(col, df_nulls_removed[col].isna().sum())
        df_nulls_removed[col].value_counts(dropna=False)

calculatedbathnbr 197
square_feet 127
finishedsquarefeet12 332
lot_size 381
regionidcity 1036
regionidneighborhood 33466
zip 27
yearbuilt 159
tax_value 1
taxamount 4
censustractandblock 121


In [30]:
df_nulls_removed[df_nulls_removed.censustractandblock.isna()].fips.value_counts(dropna=False)

6037.0    68
6059.0    34
6111.0    19
Name: fips, dtype: int64